In [1]:
from vttformatter import VttFormatter

In [2]:
filename = 'test_AutoGeneratedCaption.vtt'

In [3]:
vtt = VttFormatter(filename)
vtt.reformat_vtt()

In [4]:
import re
string = 'A1B2'
pattern = re.compile("^([A-Z][0-9]+)+$")
if pattern.match(string):
    print(True)

True


In [5]:
st = 'NOTE Confidence: 0.69450831413269'
s = st.split(':')[1].strip()
s
pattern = re.compile(r'[0-1]{1}.[0-9]{14}')
if pattern.match(s):
    print(True)

True
